In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from splinter import Browser
from io import StringIO
import time
import requests
import datetime as dt
import openpyxl
from openpyxl import workbook
from openpyxl import load_workbook


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common import keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Install new Chrome webdriver only if needed. Chrome releases new drivers approximately once a month. 
#The line below should be run whenever there is an error with the code block below. Make sure the driver is in your
#base directory. If you already have the driver, skip this line by adding a '#' and making it a comment.

#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [3]:
#Open chrome and navigate to the ACWI ETF's profile on Morningstar. We will scrape sector weightings from here.

#create webdriver
driver=webdriver.Chrome()
driver.get("https://www.morningstar.com/etfs/xnas/acwi/portfolio")


In [4]:
#You have to add time in order to give python time to load the data, otherwise you have to run the code below twice.
time.sleep(2)

In [5]:
#Scrape ACWI sector weights and labels using full XPaths     

#This code scrapes the data in the first row, one column at a time, and stores the columns values in multiple lists. 
#Those column lists are then added to a larger row list. The process repeats until all rows are scraped.

rows = 1 +len(driver.find_elements_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div/main/div/div/div[1]/section/sal-components/div/sal-components-funds-portfolio/div/div[1]/div/div[2]/div/div[6]/div[1]/div/div/div/div/div/div[2]/div[1]/div/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr'))
cols=len(driver.find_elements_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div/main/div/div/div[1]/section/sal-components/div/sal-components-funds-portfolio/div/div[1]/div/div[2]/div/div[6]/div[1]/div/div/div/div/div/div[2]/div[1]/div/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr[1]/td'))
list_1 = []
for i in range(1, rows):
    list_2 = []
    for j in range(1, cols+1):
        value = driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div[3]/div/main/div/div/div[1]/section/sal-components/div/sal-components-funds-portfolio/div/div[1]/div/div[2]/div/div[6]/div[1]/div/div/div/div/div/div[2]/div[1]/div/div/div/div/div[1]/div/div[2]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']').text
        print(value, end='     ')
        list_2.append(value)
    list_1.append(list_2)
    print()

Basic Materials     4.83     5.13     
Consumer Cyclical     10.08     10.28     
Financial Services     16.39     13.92     
Real Estate     2.71     2.70     
Communication Services     6.97     5.87     
Energy     5.63     5.04     
Industrials     10.20     14.49     
Technology     18.61     16.03     
Consumer Defensive     7.95     8.40     
Healthcare     13.48     13.46     
Utilities     3.13     4.67     


In [6]:
#Put the list into a dataframe. Dataframes can be read from or to Excel.

df=pd.DataFrame(list_1)
df

,0,1,2
0,Basic Materials,4.83,5.13
1,Consumer Cyclical,10.08,10.28
2,Financial Services,16.39,13.92
3,Real Estate,2.71,2.70
4,Communication Services,6.97,5.87
5,Energy,5.63,5.04
6,Industrials,10.20,14.49
7,Technology,18.61,16.03
8,Consumer Defensive,7.95,8.40
9,Healthcare,13.48,13.46


In [7]:
#drop the last column, rename remaining columns
df2= df.drop([2],axis=1)
df2.rename(columns={0:'GICS Sector',1:'ACWI Weight'},inplace=True)

#rename sectors and remove whitespaces in sector names to match GICS sectors
df2['GICS Sector']=df2['GICS Sector'].replace({'Consumer Cyclical':'Consumer Discretionary','Basic Materials':'Materials','Financial Services':'Financials','Technology':'Information Technology','Consumer Defensive':'Consumer Staples'})
df2



,GICS Sector,ACWI Weight
0,Materials,4.83
1,Consumer Discretionary,10.08
2,Financials,16.39
3,Real Estate,2.71
4,Communication Services,6.97
5,Energy,5.63
6,Industrials,10.20
7,Information Technology,18.61
8,Consumer Staples,7.95
9,Healthcare,13.48


In [8]:
#Get latest weightings for each holding. This code won't run unless the corresponding 'Historical Close' file has been
#created.

today = dt.datetime.today().strftime("%m.%d.%Y")
ClosePath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/GIF 2023 Brokerage data/GIF_historical_closes{today}.xlsx'

weightdf=pd.read_excel(ClosePath,sheet_name='GIF_YTD_Perf',index_col=0)

#Count number of columns. Today's market values are at index cols-6
cols=weightdf.shape[1]
cols

#Dataframe should just be ticker and current value
weightdf=weightdf.iloc[:,[0,cols-6]]

#Calculate weight of each position based on total portfolio MV
MVsum=weightdf['Latest_Total_Value'].sum()
weightdf['GIF_Weight']=weightdf['Latest_Total_Value']/MVsum
weightdf


,Ticker,Latest_Total_Value,GIF_Weight
0,GOOG,21556.529778,0.028264
1,AON,38445.440155,0.050409
2,AAPL,30958.200150,0.040592
3,BRK-B,25163.999023,0.032994
4,CAT,27756.400070,0.036393
5,CI,38477.198486,0.050450
6,DHI,20618.999863,0.027035
7,HON,13442.559570,0.017626
8,RHS,20169.739136,0.026446
9,RYE,41434.078735,0.054327


In [9]:
#Get path of cost basis file, which has GICS sectors listed. This code won't run unless the corresponding 
#'Cost Basis' file has been created.

BasisPath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/GIF 2023 Brokerage data/GIF_CostBasis_asof{today}.xlsx'

#Get GICS sector from above file. Then dataframe should just be ticker and GICS sector
basisdf=pd.read_excel(BasisPath,sheet_name='GIF_attributes-cost',index_col=0)
basisdf=basisdf.iloc[:,[1,13]]

#Combine the frames and match ticker w/ correct sector
combdf=pd.merge(weightdf,basisdf,on="Ticker",how='left')
combdf[['GICS_Sector']]=combdf[['GICS_Sector']].fillna("Cash")
combdf.rename(columns={'GICS_Sector':'GICS Sector','GIF_Weight':'GIF Weight'},inplace=True)

#Remove excess whitespace in the GICS Sector fields
combdf['GICS Sector'] = combdf['GICS Sector'].str.strip()
combdf


,Ticker,Latest_Total_Value,GIF Weight,GICS Sector
0,GOOG,21556.529778,0.028264,Communication Services
1,AON,38445.440155,0.050409,Financials
2,AAPL,30958.200150,0.040592,Information Technology
3,BRK-B,25163.999023,0.032994,Financials
4,CAT,27756.400070,0.036393,Industrials
5,CI,38477.198486,0.050450,Healthcare
6,DHI,20618.999863,0.027035,Consumer Discretionary
7,HON,13442.559570,0.017626,Industrials
8,RHS,20169.739136,0.026446,Consumer Staples
9,RYE,41434.078735,0.054327,Energy


In [10]:
#Calculate weight of each sector, drop the total value column

GIFsecdf=combdf.groupby('GICS Sector').sum()
GIFsecdf= GIFsecdf.drop(["Latest_Total_Value"],axis=1)
GIFsecdf


,GIF Weight
GICS Sector,
Cash,0.070368
Communication Services,0.049188
Consumer Discretionary,0.069996
Consumer Staples,0.098807
Energy,0.054327
Financials,0.165273
Healthcare,0.159850
Industrials,0.096856
Information Technology,0.135617


In [11]:
#Combine the GIF and ACWI weight dataframes

#Match ticker w/ correct sector, use 0 for ACWI Cash weight, cast both weights as float,
#state weights as %, compute active weight as GIF-ACWI
activedf=pd.merge(GIFsecdf,df2,on="GICS Sector",how='left')

activedf['ACWI Weight'] = activedf['ACWI Weight'].fillna(0)
activedf['ACWI Weight'] = activedf['ACWI Weight'].astype(float)
activedf['GIF Weight']=activedf['GIF Weight']*100
activedf['GIF Weight'] = activedf['GIF Weight'].astype(float)
activedf['Active Weight']=activedf['GIF Weight']-activedf['ACWI Weight']
activedf

,GICS Sector,GIF Weight,ACWI Weight,Active Weight
0,Cash,7.036845,0.00,7.036845
1,Communication Services,4.918812,6.97,-2.051188
2,Consumer Discretionary,6.999569,10.08,-3.080431
3,Consumer Staples,9.880665,7.95,1.930665
4,Energy,5.432727,5.63,-0.197273
5,Financials,16.527311,16.39,0.137311
6,Healthcare,15.984983,13.48,2.504983
7,Industrials,9.685568,10.20,-0.514432
8,Information Technology,13.561665,18.61,-5.048335
9,Materials,4.444540,4.83,-0.385460


In [12]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

#Write the file to the following location
ActivePath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/GIF 2023 Brokerage data/Active Weights/Active_Weights_asof{today}.xlsx'

#Use the ExcelWriter function within Pandas to create the file.
writer = pd.ExcelWriter(ActivePath, engine='xlsxwriter')
#Writher the activedf dataframe to this worksheet and save it.
activedf.to_excel(writer,sheet_name='Active Weights')
writer.save()

#Close the browser after scraping
driver.quit()
